In [1]:
from sklearn.utils.estimator_checks import check_estimator

from classification.OneRClassifier import OneRClassifier

In [2]:
check_estimator(OneRClassifier())

InvalidParameterError: The 'y_pred' parameter of check_estimator must be an array-like or a sparse matrix. Got Ellipsis instead.